In [5]:


# imports
import pandas as pd
import json
from pandas import json_normalize
import numpy as np
import requests
import urllib.parse
import os
from pprint import pprint



In [6]:
with open(f"{os.path.expanduser('~')}/temp-keys/fsq-primary") as f:
    fsq_key = f.readline().rstrip()

with open(f"{os.path.expanduser('~')}/temp-keys/yelp-key") as f:
    yelp_key = f.readline().rstrip()

# Get location coordinates using nominatim api


In [17]:
the_place = 'Pomona College'
# address = input("Please enter a location to query!")

def get_coords_str(place_name_or_address):
    url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(place_name_or_address) +'?format=json'

    response = requests.get(url).json()
    latitude = response[0]['lat']
    longitude = response[0]['lon']
    return latitude + "," + longitude

def get_coords_tuple(place_name_or_address):
    url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(place_name_or_address) +'?format=json'

    response = requests.get(url).json()
    latitude = response[0]['lat']
    longitude = response[0]['lon']
    return latitude,longitude

radius = 1000

# Foursquare

Send a request to Foursquare with a small radius (1000m) with the location of your choice

In [20]:
def fsq_place_query(place):
    loc_param = "ll=" + place
    rad_param= "radius=" + str(radius)
    fsq_url = f"https://api.foursquare.com/v3/places/search?"

    headers = {
    "Accept": "application/json",
    "Authorization": fsq_key
    }
    return requests.get(fsq_url + loc_param + "&" + rad_param, headers=headers)


Parse through the response to get the POI details you want (rating, name, location, etc)

In [31]:
with fsq_place_query(get_coords_str(the_place)) as fsq_response:
    fsq_data = json.loads(fsq_response.text)

# parsed = fsq_response.json()["results"]
# pd.read_json(fsq_response.text)




Put your parsed results into a DataFrame

In [22]:
context_df = json_normalize(fsq_data['context'])
main_df = json_normalize(fsq_data,"results",meta=[['context','geo_bounds','circle',],['context','geo_bounds','circle','center']],errors='ignore')
category_df = json_normalize(fsq_data['results'],"categories",meta=["fsq_id","name"], meta_prefix='place_', errors='ignore')


# Yelp

Send a request with the same location parameters (location, radius, etc)

In [27]:
def yelp_place_query(place):
    latitude,longitude = get_coords_tuple(place)
    yelp_url = "https://api.yelp.com/v3/businesses/search?"
    yelp_params = f"latitude={latitude}&longitude={longitude}&radius={radius}"


    headers = {
        "Authorization": f"Bearer {yelp_key}"
    }
    return requests.get(yelp_url + yelp_params, headers=headers)

yelp_res = yelp_place_query(the_place)

Parse through your result and get POI details

In [47]:
yelp_data = json.loads(yelp_res.text)
yelp_df_main = json_normalize(yelp_data,"businesses")
yelp_df_categories = json_normalize(yelp_data,record_path=["businesses","categories"],meta=[['businesses','id'],['businesses','name']])

Put your parsed results into a DataFrame

In [49]:
yelp_df_main

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,coordinates.latitude,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address
0,SlhHQiSpqxOk8BzCtdjK4Q,dr-grubbs-claremont-2,Dr Grubb's,https://s3-media1.fl.yelpcdn.com/bphoto/VzSUeP...,False,https://www.yelp.com/biz/dr-grubbs-claremont-2...,1138,"[{'alias': 'tradamerican', 'title': 'American ...",4.5,[delivery],...,34.096809,-117.718754,373 W Bonita Ave,None,None,Claremont,91711,US,CA,"[373 W Bonita Ave, Claremont, CA 91711]"
1,FzX6_0QFfsKFuJzys6qyPQ,the-quarter-creole-cuisine-claremont-3,The Quarter Creole Cuisine,https://s3-media2.fl.yelpcdn.com/bphoto/H17Xn8...,False,https://www.yelp.com/biz/the-quarter-creole-cu...,762,"[{'alias': 'cajun', 'title': 'Cajun/Creole'}]",4.5,"[pickup, restaurant_reservation, delivery]",...,34.095830,-117.719620,201 N Indian Hill Blvd,Ste 103,None,Claremont,91711,US,CA,"[201 N Indian Hill Blvd, Ste 103, Claremont, C..."
2,BPcFteFHJqFNIukMh2_IEQ,i-like-pie-bakeshop-claremont-2,I Like Pie Bakeshop,https://s3-media3.fl.yelpcdn.com/bphoto/Nu9cgP...,False,https://www.yelp.com/biz/i-like-pie-bakeshop-c...,837,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",4.5,"[pickup, delivery]",...,34.095513,-117.719626,175 N Indian Hill Blvd,Ste 102B,,Claremont,91711,US,CA,"[175 N Indian Hill Blvd, Ste 102B, Claremont, ..."
3,BpQkiZJAqoFGy1Ysd6hZzg,some-crust-bakery-claremont,Some Crust Bakery,https://s3-media2.fl.yelpcdn.com/bphoto/F5hRDP...,False,https://www.yelp.com/biz/some-crust-bakery-cla...,1242,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",4.0,[delivery],...,34.095078,-117.718178,119 Yale Ave,,,Claremont,91711,US,CA,"[119 Yale Ave, Claremont, CA 91711]"
4,TFxcmjVIsW5bdW7rilHHpQ,eureka-claremont-3,Eureka! Claremont,https://s3-media1.fl.yelpcdn.com/bphoto/7W9g1x...,False,https://www.yelp.com/biz/eureka-claremont-3?ad...,2954,"[{'alias': 'tradamerican', 'title': 'American ...",4.0,"[pickup, delivery]",...,34.094379,-117.721687,580 W 1st St,,,Claremont,91711,US,CA,"[580 W 1st St, Claremont, CA 91711]"
5,JLyP3zNKUb1VFD0MVj0ZxQ,the-last-drop-cafe-claremont,The Last Drop Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/4JIg5I...,False,https://www.yelp.com/biz/the-last-drop-cafe-cl...,697,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",4.5,"[pickup, delivery]",...,34.095020,-117.716428,119 Harvard Ave,None,,Claremont,91711,US,CA,"[119 Harvard Ave, Claremont, CA 91711]"
6,JlN5BhjbErkp6myx-8Adkw,kazama-sushi-claremont,Kazama Sushi,https://s3-media4.fl.yelpcdn.com/bphoto/7mJuFD...,False,https://www.yelp.com/biz/kazama-sushi-claremon...,1061,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,[delivery],...,34.095166,-117.719514,101 N Indian Hill Blvd,Ste C1-104,,Claremont,91711,US,CA,"[101 N Indian Hill Blvd, Ste C1-104, Claremont..."
7,81IjU5L-t-QQwsE38C63hQ,the-back-abbey-claremont,The Back Abbey,https://s3-media2.fl.yelpcdn.com/bphoto/3x0j0D...,False,https://www.yelp.com/biz/the-back-abbey-clarem...,2059,"[{'alias': 'burgers', 'title': 'Burgers'}, {'a...",4.0,[delivery],...,34.094939,-117.720535,128 N Oberlin Ave,,,Claremont,91711,US,CA,"[128 N Oberlin Ave, Claremont, CA 91711]"
8,LyxuqM3_4t21MPRPSaWN1A,the-whisper-house-claremont,The Whisper House,https://s3-media1.fl.yelpcdn.com/bphoto/65NFXM...,False,https://www.yelp.com/biz/the-whisper-house-cla...,1754,"[{'alias': 'newamerican', 'title': 'American (...",4.0,"[restaurant_reservation, delivery]",...,34.094573,-117.720321,502 W 1st St,,,Claremont,91711,US,CA,"[502 W 1st St, Claremont, CA 91711]"
9,LWN_UEueFvwB7whooBfPjA,bardot-claremont-2,Bardot,https://s3-media2.fl.yelpcdn.com/bphoto/hgLumF...,False,https://www.yelp.com/biz/bardot-claremont-2?ad...,1314,"[{'alias': 'newamerican', 'title': 'American (...",4.0,[delivery],...,34.096527,-117.716716,206 W Bonita Ave,,,Claremont,91711,US,CA,"[206 W Bonita Ave, Claremont, CA 91711]"


In [48]:
yelp_df_categories


,alias,title,businesses.id,businesses.name
0,tradamerican,American (Traditional),SlhHQiSpqxOk8BzCtdjK4Q,Dr Grubb's
1,cajun,Cajun/Creole,FzX6_0QFfsKFuJzys6qyPQ,The Quarter Creole Cuisine
2,bakeries,Bakeries,BPcFteFHJqFNIukMh2_IEQ,I Like Pie Bakeshop
3,desserts,Desserts,BPcFteFHJqFNIukMh2_IEQ,I Like Pie Bakeshop
4,bakeries,Bakeries,BpQkiZJAqoFGy1Ysd6hZzg,Some Crust Bakery
5,breakfast_brunch,Breakfast & Brunch,BpQkiZJAqoFGy1Ysd6hZzg,Some Crust Bakery
6,customcakes,Custom Cakes,BpQkiZJAqoFGy1Ysd6hZzg,Some Crust Bakery
7,tradamerican,American (Traditional),TFxcmjVIsW5bdW7rilHHpQ,Eureka! Claremont
8,bakeries,Bakeries,JLyP3zNKUb1VFD0MVj0ZxQ,The Last Drop Cafe
9,coffee,Coffee & Tea,JLyP3zNKUb1VFD0MVj0ZxQ,The Last Drop Cafe


# Google (stretch)

Use the same process as the first two APIs

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

Get the top 10 restaurants according to their rating

# Travelling Salesman Problem (stretch)

If you have time, follow the steps in the [ortools tutorial](https://developers.google.com/optimization/routing/tsp) using Google's [Directions API](https://developers.google.com/maps/documentation/directions/start).